# PC-AVS  Pose-Controllable Audio-Visual System Colab

[Watch this video to guide you through the process](https://www.youtube.com/watch?v=4O3EqIiEzKQ&ab_channel=bycloudump)

Original code : [visit here](https://github.com/Hangz-nju-cuhk/Talking-Face_PC-AVS)

This notebook was put together by [BibbyBob](https://github.com/BibbyBoisGitHub) and [justinjohn-03](https://github.com/justinjohn0306)

In [ ]:
#@markdown # Checks your GPU
!nvidia-smi -L

In [ ]:
#@markdown ### Download the code and install the dependencies
!git clone https://github.com/bycloudai/PCAVS-Windows
%cd /content/PCAVS-Windows
!pip install git+https://github.com/wkentaro/gdown.git
!pip install face-alignment
!pip install -r requirements.txt
!pip install lws

### **Patch** (run only once)

In [ ]:
%%writefile /content/PCAVS-Windows/scripts/align_68.py
import face_alignment
import os
import cv2
import skimage.transform as trans
import argparse
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'


def get_affine(src):
    dst = np.array([[87,  59],
                    [137,  59],
                    [112, 120]], dtype=np.float32)
    tform = trans.SimilarityTransform()
    tform.estimate(src, dst)
    M = tform.params[0:2, :]
    return M


def affine_align_img(img, M, crop_size=224):
    warped = cv2.warpAffine(img, M, (crop_size, crop_size), borderValue=0.0)
    return warped


def affine_align_3landmarks(landmarks, M):
    new_landmarks = np.concatenate([landmarks, np.ones((3, 1))], 1)
    affined_landmarks = np.matmul(new_landmarks, M.transpose())
    return affined_landmarks


def get_eyes_mouths(landmark):
    three_points = np.zeros((3, 2))
    three_points[0] = landmark[36:42].mean(0)
    three_points[1] = landmark[42:48].mean(0)
    three_points[2] = landmark[60:68].mean(0)
    return three_points


def get_mouth_bias(three_points):
    bias = np.array([112, 120]) - three_points[2]
    return bias


def align_folder(folder_path, folder_save_path):

    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device=device)
    preds = fa.get_landmarks_from_directory(folder_path)

    sumpoints = 0
    three_points_list = []

    for img in preds.keys():
        pred_points = np.array(preds[img])
        if pred_points is None or len(pred_points.shape) != 3:
            print('preprocessing failed')
            return False
        else:
            num_faces, size, _ = pred_points.shape
            if num_faces == 1 and size == 68:

                three_points = get_eyes_mouths(pred_points[0])
                sumpoints += three_points
                three_points_list.append(three_points)
            else:

                print('preprocessing failed')
                return False
    avg_points = sumpoints / len(preds)
    M = get_affine(avg_points)
    p_bias = None
    for i, img_pth in enumerate(preds.keys()):
        three_points = three_points_list[i]
        affined_3landmarks = affine_align_3landmarks(three_points, M)
        bias = get_mouth_bias(affined_3landmarks)
        if p_bias is None:
            bias = bias
        else:
            bias = p_bias * 0.2 + bias * 0.8
        p_bias = bias
        M_i = M.copy()
        M_i[:, 2] = M[:, 2] + bias
        img = cv2.imread(img_pth)
        wrapped = affine_align_img(img, M_i)
        img_save_path = os.path.join(folder_save_path, img_pth.split('/')[-1])
        cv2.imwrite(img_save_path, wrapped)
    print('cropped files saved at {}'.format(folder_save_path))


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--folder_path', help='the folder which needs processing')
    args = parser.parse_args()

    if os.path.isdir(args.folder_path):
        home_path = '/'.join(args.folder_path.split('/')[:-1])
        save_img_path = os.path.join(home_path, args.folder_path.split('/')[-1] + '_cropped')
        os.makedirs(save_img_path, exist_ok=True)

        align_folder(args.folder_path, save_img_path)


if __name__ == '__main__':
    main()

# **Process**

In [ ]:
#@markdown #### Download the checkpoints and unzip them
import gdown
!gdown 1Zehr3JLIpzdg2S5zZrhIbpYPKF-4gKU_
!unzip demo.zip -d /content/PCAVS-Windows/checkpoints

In [ ]:
#@markdown #### Unzips the misc files in misc folder 
!unzip /content/PCAVS-Windows/misc/Audio_Source.zip -d /content/PCAVS-Windows/misc/
!rm /content/PCAVS-Windows/misc/Audio_Source.zip
!unzip /content/PCAVS-Windows/misc/Input.zip -d /content/PCAVS-Windows/misc/
!rm /content/PCAVS-Windows/misc/Input.zip
!unzip /content/PCAVS-Windows/misc/Mouth_Source.zip -d /content/PCAVS-Windows/misc/
!rm /content/PCAVS-Windows/misc/Mouth_Source.zip
!unzip /content/PCAVS-Windows/misc/Pose_Source.zip -d /content/PCAVS-Windows/misc/
!rm /content/PCAVS-Windows/misc/Pose_Source.zip
%cd /content/PCAVS-Windows

In [ ]:
#@markdown ### This cell extracts the frames from the video
#@markdown make sure to create the folder and edit this cell accodringly before running
!ffmpeg -i misc/Pose_Source/trump.mp4 -vf fps=24 misc/Pose_Source/trump/%06d.jpg

### The cells below will align the frames extracted from both the video and image 
(the longer the video is, the longer the process)

In [ ]:
!python scripts/align_68.py --folder_path misc/Input/biden

In [ ]:
!python scripts/align_68.py --folder_path misc/Pose_Source/trump

### This is the final cell that you have to run, after editing the ``demo.csv`` file, run the cell below and get the results in the ``results`` folder.

In [ ]:
!python inference.py --name demo --meta_path_vox misc/demo.csv --netG modulate --netA resseaudio --netA_sync ressesync --netD multiscale --netV resnext --netE fan --model av --gpu_ids 0 --clip_len 1 --batchSize 16 --style_dim 2560 --nThreads 4 --input_id_feature --generate_interval 1 --style_feature_loss --use_audio 1 --noise_pose --gen_video --driving_pose --generate_from_audio_only